<a href="https://colab.research.google.com/github/ayan-2025/Learning-to-Learn-Model-Regression-Network/blob/main/Learning_to_Learn_Model_Regression_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive/')
import warnings
warnings.filterwarnings('ignore')

%cd /content/drive/My Drive/Colab Notebooks/


data_set = pd.read_csv("Data_Q2.csv")
data_set.head()

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks


,Temperature,Humidity,Wind Speed,Flow,Consumption
0,5.578,93.00,0.082,0.185,5935.174070
1,15.510,64.38,0.085,0.133,6044.657863
2,15.730,64.21,0.084,0.152,6061.944778
3,15.620,65.22,0.083,0.145,6108.043217
4,15.450,67.69,0.083,0.189,6119.567827


In [2]:
frame2 = pd.read_csv('Data_Q2.csv')

def label_data(row):
    if row['Consumption'] <= 6500:
        return 1
    elif row['Consumption'] > 6500 and row['Consumption'] <= 7000:
        return 2
    elif row['Consumption'] > 7000 and row['Consumption'] <= 7500:
        return 3
    elif row['Consumption'] > 7500 and row['Consumption'] <= 8000:
        return 4
    elif row['Consumption'] > 8000 and row['Consumption'] <= 8500:
        return 5
    elif row['Consumption'] > 8500 and row['Consumption'] <= 9000:
        return 6
    elif row['Consumption'] > 9000:
         return 7    

frame2['Class'] = frame2.apply(lambda row: label_data(row), axis=1)

frame2

,Temperature,Humidity,Wind Speed,Flow,Consumption,Class
0,5.578,93.00,0.082,0.185,5935.174070,1
1,15.510,64.38,0.085,0.133,6044.657863,1
2,15.730,64.21,0.084,0.152,6061.944778,1
3,15.620,65.22,0.083,0.145,6108.043217,1
4,15.450,67.69,0.083,0.189,6119.567827,1
...,...,...,...,...,...,...
995,17.330,42.24,4.917,31.540,9443.855422,7
996,7.010,76.40,4.920,65.890,9449.638554,7
997,14.810,82.30,4.913,0.159,9449.638554,7
998,12.090,77.40,0.073,0.104,9449.638554,7


In [3]:
#check class imbalance issue
class1_data = frame2[frame2['Class'] == 1] 
class2_data = frame2[frame2['Class'] == 2] 
class3_data = frame2[frame2['Class'] == 3] 
class4_data = frame2[frame2['Class'] == 4] 
class5_data = frame2[frame2['Class'] == 5] 
class6_data = frame2[frame2['Class'] == 6] 
class7_data = frame2[frame2['Class'] == 7] 

print( len(class1_data), len(class2_data), len(class3_data), len(class4_data), len(class5_data), len(class6_data), len(class7_data))

15 27 103 218 280 255 102


In [4]:
from sklearn.preprocessing import StandardScaler

X = frame2.drop("Class", axis=1)

scaler = StandardScaler()

groups = frame2.groupby("Class")

for label, group in groups:
  X_group = group.drop("Class", axis=1)
  X_group_std = scaler.fit_transform(X_group)
  X.loc[X_group.index, :] = X_group_std

frame2_std = pd.concat([X, frame2["Class"]], axis=1)
frame2_std

,Temperature,Humidity,Wind Speed,Flow,Consumption,Class
0,-1.690575,1.563325,-0.508727,-0.365626,-1.925782,1
1,1.083295,-1.525195,0.581402,-0.412271,-1.304210,1
2,1.144738,-1.543541,0.218026,-0.395227,-1.206067,1
3,1.114017,-1.434547,-0.145350,-0.401507,-0.944353,1
4,1.066538,-1.167998,-0.145350,-0.362038,-0.878924,1
...,...,...,...,...,...,...
995,0.572096,-2.357184,1.585485,-0.112966,1.711956,7
996,-1.984162,0.416500,1.586861,0.552813,1.753338,7
997,-0.052107,0.895561,1.583650,-0.721200,1.753338,7
998,-0.725849,0.497697,-0.636763,-0.722266,1.753338,7


In [5]:
X = frame2_std[['Temperature','Humidity','Wind Speed','Flow']]
y = frame2_std[['Class']]

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

print(len(y_resampled[y_resampled['Class'] == 1]), len(y_resampled[y_resampled['Class'] == 2]), len(y_resampled[y_resampled['Class'] == 3]), len(y_resampled[y_resampled['Class'] == 4]), len(y_resampled[y_resampled['Class'] == 5]), len(y_resampled[y_resampled['Class'] == 6]), len(y_resampled[y_resampled['Class'] == 7]))

280 280 280 280 280 280 280


In [6]:
new_data_set = np.concatenate((X_resampled, y_resampled), axis=1)
new_data_set = pd.DataFrame(new_data_set)
new_data_set.columns = ['Temperature','Humidity','Wind Speed','Flow','Class']
new_data_set

,Temperature,Humidity,Wind Speed,Flow,Class
0,-1.690575,1.563325,-0.508727,-0.365626,1.0
1,1.083295,-1.525195,0.581402,-0.412271,1.0
2,1.144738,-1.543541,0.218026,-0.395227,1.0
3,1.114017,-1.434547,-0.145350,-0.401507,1.0
4,1.066538,-1.167998,-0.145350,-0.362038,1.0
...,...,...,...,...,...
1955,0.520079,1.488298,1.589155,-0.721917,7.0
1956,0.809887,-0.607392,-0.630799,0.153733,7.0
1957,0.215409,-0.190040,1.587779,-0.463785,7.0
1958,0.572096,-2.357184,1.585485,-0.112966,7.0


Now we are making Model pair set 

In [7]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split as split
from sklearn.utils import resample

model_X = []
model_y = []

for i in range(1,8):
   
   negative_examples = X_resampled[y_resampled['Class'] != i]
   num_negative_examples = 100
   negative_examples_subsampled = resample(negative_examples, n_samples=num_negative_examples, random_state=42)
   
   subsampled_X = np.vstack((X_resampled[y_resampled['Class'] == i], negative_examples_subsampled))
   subsampled_y = np.hstack((np.ones(np.count_nonzero(y_resampled['Class'] == i)), np.zeros(num_negative_examples)))
   
   random_state_C_map = {0: 0.01, 42: 0.1, 123: 1, 130: 10, 140: 100}
   
   train_X, test_X, train_y, test_y = split(subsampled_X, subsampled_y, test_size=0.1, shuffle=True, random_state=0)
   param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
   svm = SVC(kernel='linear')
   grid_search = GridSearchCV(svm, param_grid, cv=5)
   grid_search.fit(train_X, train_y)
   C = grid_search.best_params_['C']
   svm = SVC(kernel='linear', C=C)
   svm.fit(train_X, train_y)
   w_1 = svm.coef_

   print("For class: ", i)
   print("Best parameters:", grid_search.best_params_)
   print("Accuracy:", grid_search.score(test_X, test_y))
   print("w_1 value: ", w_1)

   for random_state in random_state_C_map.keys():
     train_small_X, test_small_X, train_small_y, test_small_y = split(train_X, train_y, test_size=0.1, shuffle=True, random_state=random_state)
     C = random_state_C_map[random_state]
     svm = SVC(kernel='linear', C=C)
     svm.fit(train_small_X, train_small_y)
     w_0 = svm.coef_
     model_X.append(w_0)
     model_y.append(w_1)



For class:  1
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 value:  [[ 2.36030199e-04  2.84319845e-04 -3.23671224e-05 -7.61957419e-05]]
For class:  2
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 value:  [[ 1.30108352e-04  5.75706266e-05 -3.16517844e-05  1.12025210e-04]]
For class:  3
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 value:  [[2.42240805e-04 6.49448912e-05 3.32207662e-05 5.48379781e-05]]
For class:  4
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 value:  [[ 3.86417237e-06 -2.72897475e-06 -4.08114292e-05  1.58702480e-06]]
For class:  5
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 value:  [[-8.81307193e-05  3.65247840e-06 -2.65497577e-04 -1.57187313e-05]]
For class:  6
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 value:  [[ 1.12683832e-04 -3.60236961e-06  2.91432730e-05 -9.83424378e-05]]
For class:  7
Best parameters: {'C': 0.0001}
Accuracy: 0.7368421052631579
w_1 va

Models learned from small-sample set for all classes (1 to 7)

In [8]:
model_X = np.array(model_X)
model_X = model_X.reshape(35,4)
df = pd.DataFrame(model_X)
df.columns = ['w_01','w_02','w_03','w_04']
df 

,w_01,w_02,w_03,w_04
0,8.417762e-04,9.189898e-04,-0.000011,-7.510900e-05
1,7.071830e-04,6.672086e-04,0.000136,-1.699227e-05
2,5.217267e-04,5.009202e-04,0.000110,-2.885179e-07
3,6.781229e-04,7.718607e-04,0.000109,3.468925e-05
4,4.198607e-04,4.554498e-04,0.000286,-6.910577e-06
5,1.857001e-04,8.495706e-05,-0.000152,-8.334554e-05
6,4.940747e-05,3.842763e-05,0.000070,-1.217857e-05
7,1.607051e-04,-6.365120e-05,-0.000096,-1.183992e-04
8,6.163024e-05,9.626130e-05,-0.000190,-6.076410e-05
9,3.499741e-05,-7.801793e-05,-0.000100,-4.624893e-05


Models learned from large-sample set for all classes (1 to 7)

In [9]:
model_y = np.array(model_y)
model_y = model_y.reshape(35,4)
df_ = pd.DataFrame(model_y)
df_.columns = ['w_11','w_12','w_13','w_14']
df_

,w_11,w_12,w_13,w_14
0,0.000236,0.000284,-0.000032,-0.000076
1,0.000236,0.000284,-0.000032,-0.000076
2,0.000236,0.000284,-0.000032,-0.000076
3,0.000236,0.000284,-0.000032,-0.000076
4,0.000236,0.000284,-0.000032,-0.000076
5,0.000130,0.000058,-0.000032,0.000112
6,0.000130,0.000058,-0.000032,0.000112
7,0.000130,0.000058,-0.000032,0.000112
8,0.000130,0.000058,-0.000032,0.000112
9,0.000130,0.000058,-0.000032,0.000112


In [10]:
X_train, X_test, y_train, y_test = split(model_X, model_y, test_size=0.1, shuffle=True, random_state=0)

In [17]:
import tensorflow as tf
from tensorflow import keras

d = df.shape[1]

model = keras.Sequential([
    keras.layers.Dense(90, input_shape=(d,), activation='relu'),
    keras.layers.Dense(45, activation='relu'),
    #keras.layers.Dense(32, activation='relu'),
    #keras.layers.Dense(32, activation='relu'),
    #keras.layers.Dense(32, activation='relu'),
    #keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(d)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
1/1 [==============================] - 1s 540ms/step - loss: 1.2399e-08 - val_loss: 5.4872e-06
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 5.2983e-06 - val_loss: 1.0298e-07
Epoch 3/10
1/1 [==============================] - 0s 25ms/step - loss: 1.0602e-07 - val_loss: 5.6006e-07
Epoch 4/10
1/1 [==============================] - 0s 23ms/step - loss: 5.8760e-07 - val_loss: 9.1283e-07
Epoch 5/10
1/1 [==============================] - 0s 23ms/step - loss: 9.7005e-07 - val_loss: 7.4626e-07
Epoch 6/10
1/1 [==============================] - 0s 27ms/step - loss: 8.1914e-07 - val_loss: 4.2240e-07
Epoch 7/10
1/1 [==============================] - 0s 28ms/step - loss: 4.8679e-07 - val_loss: 1.4058e-07
Epoch 8/10
1/1 [==============================] - 0s 28ms/step - loss: 1.7487e-07 - val_loss: 1.0730e-07
Epoch 9/10
1/1 [==============================] - 0s 26ms/step - loss: 1.0266e-07 - val_loss: 2.4894e-07
Epoch 10/10
1/1 [==============================] - 0s 

In [18]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)

acc = r2_score(y_test, y_pred)
print('Accuracy:', acc)

1/1 [==============================] - 0s 42ms/step
Accuracy: -15028.93577698079
